# Week 3 The Prep School - Passing Grades

### Import data

1. Import pandas

In [1]:
import pandas as pd

2. Import students data

In [3]:
students = pd.read_csv('PD 2022 Wk 1 Input - Input.csv')
students

,id,pupil first name,pupil last name,gender,Date of Birth,Parental Contact Name_1,Parental Contact Name_2,Preferred Contact Employer,Parental Contact
0,1,Ronna,Nellies,Female,12/21/2013,Purcell,Ketti,Demizz,1
1,2,Rusty,Andriulis,Male,7/21/2012,Vassili,Rivi,Brainbox,1
2,3,Roberta,Oakeshott,Female,12/4/2011,Lind,Haskell,Centidel,2
3,4,Lola,Rubinfajn,Male,6/29/2012,Elie,Tresa,Edgeblab,2
4,5,Kamila,Benedtti,Female,7/10/2012,Adela,Clevey,Trudoo,1
...,...,...,...,...,...,...,...,...,...
995,996,Ninetta,Worling,Female,2/15/2015,Sibbie,Peterus,Omba,2
996,997,Stanford,Tinton,Female,4/9/2013,Kyle,Augustin,Dabshots,1
997,998,Ertha,MacCook,Male,12/14/2013,Sidonia,Kessiah,Topicshots,2
998,999,Lawton,Randles,Female,12/12/2011,Kali,Reeva,Skalith,1


3. Import grades data

In [4]:
grades = pd.read_csv('PD 2022 WK 3 Grades.csv')
grades

,Student ID,Maths,English,Spanish,Science,Art,History,Geography
0,1,66,97,85,75,76,94,76
1,2,84,85,62,87,68,75,74
2,3,88,68,69,81,92,89,75
3,4,65,97,96,89,98,77,62
4,5,86,97,94,98,67,77,97
...,...,...,...,...,...,...,...,...
995,996,100,85,83,91,91,95,64
996,997,97,96,87,80,97,80,93
997,998,86,89,77,94,65,73,84
998,999,63,65,87,76,98,60,69


### Remove unnecessary columns from students table

In [5]:
students = students[['id', 'gender']]

### Reshape grades table so that each student and subject has a single grade column
1. Reshape using pd.melt

In [6]:
grades = grades.melt(id_vars = ['Student ID'], value_vars = ['Maths', 'English', 'Spanish', 'Science', 'Art', 'History', 'Geography'])
grades.head()

,Student ID,variable,value
0,1,Maths,66
1,2,Maths,84
2,3,Maths,88
3,4,Maths,65
4,5,Maths,86


2. Rename columns to 'Subject' and 'Score'

In [7]:
grades = grades.rename(columns={'variable' : 'Subject', 'value' : 'Score'})
grades.head()

,Student ID,Subject,Score
0,1,Maths,66
1,2,Maths,84
2,3,Maths,88
3,4,Maths,65
4,5,Maths,86


### Find the average score for each student in all subjects
1. Create a groupby on student id
    - use chain functioning to assign student ID to columns rather than as hierarchical index
    - use chain functioning to round the average to 1 decimal place

In [8]:
avg_score = grades.groupby('Student ID').mean().reset_index().round(1)
avg_score.head()

,Student ID,Score
0,1,81.3
1,2,76.4
2,3,80.3
3,4,83.4
4,5,88.0


2. Rename the Score Column

In [20]:
avg_score = avg_score.rename(columns={'Score' : "Student's Average Score"})
avg_score.head()

,Student ID,Student's Average Score
0,1,81.3
1,2,76.4
2,3,80.3
3,4,83.4
4,5,88.0


### For each score, create a column to determine if students passed or failed
1. Import numpy so we can use np.nan in formula



In [21]:
import numpy as np

2. Create a function to assess the conditional value

In [22]:
def pf(s):
    if s >= 75:
        return 1
    else:
        return np.nan

3. Use df.apply to apply the function to the dataframe

In [23]:
grades['Pass/Fail'] = grades['Score'].apply(pf)

### Count how many subjects each student passed
1. Create a groupby to count subjects passed per student


In [24]:
subjects_passed = grades.groupby('Student ID').count().reset_index()
subjects_passed.head()

,Student ID,Subject,Score,Pass/Fail
0,1,7,7,6
1,2,7,7,4
2,3,7,7,5
3,4,7,7,5
4,5,7,7,6


2. Drop 'extra' columns to get only the column containing the correct answer

In [25]:
subjects_passed = subjects_passed[['Student ID', 'Pass/Fail']]
subjects_passed.head()

,Student ID,Pass/Fail
0,1,6
1,2,4
2,3,5
3,4,5
4,5,6


3. Rename columns to match naming standards

In [26]:
subjects_passed = subjects_passed.rename(columns={'Pass/Fail' : 'Passed Subjects'})
subjects_passed.head()

,Student ID,Passed Subjects
0,1,6
1,2,4
2,3,5
3,4,5
4,5,6


### Join all three tables to create the final table with student id, gender, average score in all subjects, and number of subjects passed
1. Join avg_score with subjects_passed

In [28]:
grades_summary = pd.merge(subjects_passed, avg_score, on='Student ID')
grades_summary

,Student ID,Passed Subjects,Student's Average Score
0,1,6,81.3
1,2,4,76.4
2,3,5,80.3
3,4,5,83.4
4,5,6,88.0
...,...,...,...
995,996,6,87.0
996,997,7,90.0
997,998,5,81.1
998,999,3,74.0


2. Join the new grades summary table with the student_id table

In [29]:
final = pd.merge(grades_summary, students, left_on='Student ID', right_on='id')
final.head()

,Student ID,Passed Subjects,Student's Average Score,id,gender
0,1,6,81.3,1,Female
1,2,4,76.4,2,Male
2,3,5,80.3,3,Female
3,4,5,83.4,4,Male
4,5,6,88.0,5,Female


### Remove duplicate columns

In [30]:
final = final[['Student ID', 'Passed Subjects', "Student's Average Score", 
       'gender']]

### Export to csv

In [31]:
final.to_csv('pandas_solution.csv', index=False)